# SECOND STEP: ENCODING

You now want to encode with a GPU (choose your model) ! go to google Colab
4 things:
- submission_metadata (all the papers) into papers.pt
- all the authors (une archive.zip 22 on doit avoir 96k et pas 70k smth) into authors.pt
- Malicious (natural_ranking_101_examples.jsonl['modified_abstract']) into malicious.pt
- reecrit.csv into rewritten.pt
- reecrit_TOTAL.csv into rewritten_all_papers


In [ ]:
!pip install -U adapters

from adapters import AutoAdapterModel
from transformers import AutoTokenizer
import torch
model = AutoAdapterModel.from_pretrained("allenai/specter2_base")
adapter_name = model.load_adapter("allenai/specter2", source="hf", set_active=True)
tokenizer = AutoTokenizer.from_pretrained('allenai/specter2_base')


#load the adapter(s) as per the required task, provide an identifier for the adapter in load_as argument and activate it
model.load_adapter("allenai/specter2", source="hf", load_as="specter2", set_active=True)


hi


In [3]:
import pandas as pd
dataset=pd.read_json('submission_metadata.json').T
dataset.index.name = 'row_id'
dataset

,title,abstract,authorsids
row_id,,,
zyhxRc9bew,What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra..."
zyZkaqNnpa,Don’t blame Dataset Shift! Shortcut Learning d...,Common explanations for shortcut learning assu...,"[~Aahlad_Manas_Puli1, ~Lily_H_Zhang1, ~Yoav_Wa..."
zuXyQsXVLF,Enhancing Adversarial Contrastive Learning via...,Adversarial contrastive learning (ACL) is a te...,"[~Xilie_Xu1, ~Jingfeng_Zhang1, ~Feng_Liu2, ~Ma..."
ztDxO15N7f,An Optimization-based Approach To Node Role Di...,"Similar to community detection, partitioning t...","[~Michael_Scholkemper1, ~Michael_T_Schaub1]"
zsOOqjaj2z,Generator Identification for Linear SDEs with ...,"In this paper, we present conditions for ident...","[~Yuanyuan_Wang5, ~Xi_Geng1, ~Wei_Huang8, ~Biw..."
...,...,...,...
08hStXdT1s,Knowledge Diffusion for Distillation,The representation gap between teacher and stu...,"[~Tao_Huang5, ~Yuan_Zhang20, ~Mingkai_Zheng1, ..."
05P1U0jk8r,Exploiting hidden structures in non-convex gam...,A wide array of modern machine learning applic...,"[~Iosif_Sakos1, ~Emmanouil-Vasileios_Vlatakis-..."
02Uc0G2Cym,Robustness Guarantees for Adversarially Traine...,We study robust adversarial training of two-la...,"[~Poorya_Mianjy1, ~Raman_Arora1]"


In [ ]:
subset_dataset = dataset

papers = subset_dataset

# concatenate title and abstract
text_batch = [d['title'] + tokenizer.sep_token + (d.get('abstract') or '') for index,d in papers.iterrows()]

# preprocess the input
maliste=[]
i=0

model.eval()

# 2. Disable gradient calculation (saves Memory)
with torch.no_grad():
  for elt in text_batch:
    print(i)
    i+=1
    inputs = tokenizer(elt, padding=True, truncation=True,
                                    return_tensors="pt", return_token_type_ids=False, max_length=512)
    output = model(**inputs)
  # take the first token in the batch as the embedding
    maliste.append(output.last_hidden_state[:, 0, :].to('cpu'))
    del inputs,output

In [ ]:
import torch
print(len(maliste))
stacked = torch.stack(maliste, dim=0)
stacked.shape
stacked=stacked.squeeze(1)
print(stacked.shape)
torch.save(stacked,'papers.pt')